In [1]:
import netCDF4 as nc
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
pd.options.mode.chained_assignment = None  # default='warn'


# Helping functions

In [2]:
def get_neighbours(df,x,y):
    
    result = pd.DataFrame()
    
    df_cell = df.loc[(df["x"] ==x) & (df["y"] ==y)]
    
    
    new_row_right = df.loc[(df["x"] ==x+1) & (df["y"] ==y)]
   #print(new_row_right)
    if(len(new_row_right) == 0):
        new_row_right = pd.DataFrame({"x" : [x+1 for i in range(df_cell.shape[0])],
                                      "y" : [y for i in range(df_cell.shape[0])],
                                      "emission_value" : [0 for i in range(df_cell.shape[0])],
                                      "hour" : df_cell["hour"].values ,
                                      "nb_type" : ["nbr" for i in range(df_cell.shape[0])]})
    else:
        new_row_right["nb_type"] = "nbr"
    
    
    new_row_left = df.loc[(df["x"] ==x-1) & (df["y"] ==y)]
    if(len(new_row_left) == 0):
        new_row_left = pd.DataFrame({"x" : [x-1 for i in range(df_cell.shape[0])],
                                      "y" : [y for i in range(df_cell.shape[0])],
                                      "emission_value" : [0 for i in range(df_cell.shape[0])],
                                      "hour" : df_cell["hour"].values ,
                                      "nb_type" : ["nbl" for i in range(df_cell.shape[0])]})
    else:
        new_row_left["nb_type"] = "nbl"
    
    
    new_row_up = df.loc[(df["x"] ==x) & (df["y"] ==y-1)]
    if(len(new_row_up) == 0):
        new_row_up = pd.DataFrame({"x" : [x for i in range(df_cell.shape[0])],
                                      "y" : [y-1 for i in range(df_cell.shape[0])],
                                      "emission_value" : [0 for i in range(df_cell.shape[0])],
                                      "hour" : df_cell["hour"].values ,
                                      "nb_type" : ["nbu" for i in range(df_cell.shape[0])]})
    else:
        new_row_up["nb_type"] = "nbu"
    
    
    new_row_down = df.loc[(df["x"] ==x) & (df["y"] ==y+1)]
    if(len(new_row_down) == 0):
        new_row_down = pd.DataFrame({"x" : [x for i in range(df_cell.shape[0])],
                                      "y" : [y+1 for i in range(df_cell.shape[0])],
                                      "emission_value" : [0 for i in range(df_cell.shape[0])],
                                      "hour" : df_cell["hour"].values ,
                                      "nb_type" : ["nbd" for i in range(df_cell.shape[0])]})
    else:
        new_row_down["nb_type"] = "nbd"
    result = pd.concat([new_row_right,new_row_left,new_row_up,new_row_down])
    return result



def add_neighbours(df,x,y):
    '''
    
        df_cell: adds neighbours to specific cell
    '''
    df_cell = df.loc[(df["x"] ==x) & (df["y"] ==y)]
    
    neighbours = get_neighbours(df,x,y)
    
    neighbours_right = neighbours.loc[neighbours["nb_type"] == "nbr"]
    neighbours_left = neighbours.loc[neighbours["nb_type"] == "nbl"]
    neighbours_up = neighbours.loc[neighbours["nb_type"] == "nbu"]
    neighbours_down = neighbours.loc[neighbours["nb_type"] == "nbd"]
    
    try:
        df_to_append = pd.DataFrame({"nbr":neighbours_right["emission_value"].values,
                                    "nbl":neighbours_left["emission_value"].values,
                                    "nbu":neighbours_up["emission_value"].values,
                                    "nbd":neighbours_down["emission_value"].values})
    except:
        df_to_append = pd.DataFrame()
        
    df_to_append = df_to_append.reset_index(drop=True)
    df_cell = df_cell.reset_index(drop=True)
    result= pd.concat([df_cell,df_to_append],axis = 1)
    return result

def complete_df_with_neighbours(df):
    '''
        
    '''
    maximal_x = max(df["x"])
    minimal_x = min(df["x"])
    maximal_y = max(df["y"])
    minimal_y = min(df["y"])
    #print(df.shape[0])
    result = pd.DataFrame()
    for i in range(minimal_x,maximal_x+1,1):
        for j in range(minimal_y,maximal_y+1):
            df_cell = add_neighbours(df,i ,j )
            #print(df_cell)
            result = pd.concat([result,df_cell])
    return result
    

## Optimizing the functions

### by another call functions

In [3]:

def get_neighbours_optimized(df,x,y):
    result = pd.DataFrame()

    # Define neighbor coordinates
    neighbors_coords = [(x+1, y, "nbr"), (x-1, y, "nbl"), (x, y-1, "nbu"), (x, y+1, "nbd")]

    for nx, ny, nb_type in neighbors_coords:
        new_row = df.loc[(df["x"] == nx) & (df["y"] == ny)].copy()

        if len(new_row) == 0:
            new_row = pd.DataFrame({
                "x": [nx] * df.shape[0],
                "y": [ny] * df.shape[0],
                "emission_value": [0] * df.shape[0],
                "hour": df["hour"].values,
                "nb_type": [nb_type] * df.shape[0]
            })
        else:
            new_row["nb_type"] = nb_type

        result = pd.concat([result, new_row])

    return result.reset_index(drop=True)



def add_neighbours_optimized(df,x,y):
    '''
    
        df_cell: adds neighbours to specific cell
    '''
    df_cell = df[(df["x"] == x) & (df["y"] == y)]

    neighbours = get_neighbours_optimized(df_cell, x, y)

    neighbours_right = neighbours[neighbours["nb_type"] == "nbr"]
    neighbours_left = neighbours[neighbours["nb_type"] == "nbl"]
    neighbours_up = neighbours[neighbours["nb_type"] == "nbu"]
    neighbours_down = neighbours[neighbours["nb_type"] == "nbd"]

    if not neighbours_right.empty:
        df_cell.loc[:, "nbr"] = neighbours_right["emission_value"].values
    if not neighbours_left.empty:
        df_cell.loc[:, "nbl"] = neighbours_left["emission_value"].values
    if not neighbours_up.empty:
        df_cell.loc[:, "nbu"] = neighbours_up["emission_value"].values
    if not neighbours_down.empty:
        df_cell.loc[:, "nbd"] = neighbours_down["emission_value"].values

    return df_cell.reset_index(drop=True)

def complete_df_with_neighbours_optimized(df):
    '''
        
    '''
    maximal_x = df["x"].max()
    minimal_x = df["x"].min()
    maximal_y = df["y"].max()
    minimal_y = df["y"].min()

    result = []

    for i in range(minimal_x, maximal_x + 1):
        for j in range(minimal_y, maximal_y + 1):
            df_cell = add_neighbours_optimized(df, i, j)
            result.append(df_cell)

    return pd.concat(result, ignore_index=True)

In [4]:
#get_neighbours(result_nspecies,600,600)

In [5]:
#get_neighbours_optimized2(result_nspecies,600,600)


### by another get neighbours return value logic

In [33]:
from line_profiler import profile
@profile
def get_neighbours_optimized2(df,x,y):
    
    result = pd.DataFrame()

    # Define neighbor coordinates
    neighbors_coords = [(x+1, y, "nbr"), (x-1, y, "nbl"), (x, y-1, "nbu"), (x, y+1, "nbd")]
    
    subdf = df[(df["x"] == x) & (df["y"] == y)]
    subdf_hours = set(subdf["hour"])
    #print(subdf)
    for nx, ny, nb_type in neighbors_coords:
        new_row = df[(df["x"] == nx) & (df["y"] == ny)]
        
        if len(new_row) == 0:
            subdf[nb_type]  = [0]*subdf.shape[0]
        else:
            subdf[nb_type] = new_row["emission_value"].values
            #print(nb_type,new_row["emission_value"])

        

    return subdf



def complete_df_with_neighbours_optimized2(df):
    '''
        
    '''
    maximal_x = df["x"].max()
    minimal_x = df["x"].min()
    maximal_y = df["y"].max()
    minimal_y = df["y"].min()

    result = []

    for i in range(minimal_x, maximal_x + 1):
        for j in range(minimal_y, maximal_y + 1):
            df_cell = get_neighbours_optimized2(df, i, j)
            result.append(df_cell)

    return pd.concat(result, ignore_index=True)

In [69]:
species1 = pd.read_csv("../data/result_species_500/result1_species_in.csv",sep=";")
species1 = species1[(species1["x"]<50) & (species1["y"]<50)]
species1



,x,y,hour,building_height,emission_value,nbr,nbl,nbu,nbd
0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1178395,49,49,43,0.0,0.0,0.0,0.0,0.0,0.0
1178396,49,49,44,0.0,0.0,0.0,0.0,0.0,0.0
1178397,49,49,45,0.0,0.0,0.0,0.0,0.0,0.0
1178398,49,49,46,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
%reload_ext line_profiler

In [37]:
%lprun -f complete_df_with_neighbours_optimized2 -f get_neighbours_optimized2 complete_df_with_neighbours_optimized2(species1)

In [ ]:
res = complete_df_with_neighbours_optimized2(species1)
res

### After Profiling

In [64]:
@profile
def get_neighbours_optimized2_profiled(df,ds, x, y,xs,ys,hours):
    result = pd.DataFrame()

    # Define neighbor coordinates
    neighbors_coords = [(x + 1, y, "nbr"), (x - 1, y, "nbl"), (x, y - 1, "nbu"), (x, y + 1, "nbd")]

    subdf = df[(df["x"] == x) & (df["y"] == y)]

    for nx, ny, nb_type in neighbors_coords:
        emission_list = []
        if(nx not in xs
                or ny not in ys):
            emission_list = [0]*len(hours)
            continue


        emission_list=ds["emission_values"][hours, 0, ny, nx, 0].data

        subdf[nb_type] = emission_list
            # print(nb_type,new_row["emission_value"])

    return subdf

@profile
def complete_df_with_neighbours_optimized2_profiled(df,ds):
    '''

    '''
    maximal_x = df["x"].max()
    minimal_x = df["x"].min()
    maximal_y = df["y"].max()
    minimal_y = df["y"].min()

    result = []

    xs = set(df["x"])
    ys = set(df["y"])
    hours = list(set(df["hour"]))
    for i in range(minimal_x, maximal_x + 1):
        for j in range(minimal_y, maximal_y + 1):
            df_cell = get_neighbours_optimized2_profiled(df,ds, i, j,xs,ys,hours)
            result.append(df_cell)

    return pd.concat(result, ignore_index=True)

In [65]:
ds = nc.Dataset("../data/photoshade_6km10m_lod2_chemistry.nc")


In [72]:
complete_df_with_neighbours_optimized2_profiled(species1,ds)

,x,y,hour,building_height,emission_value,nbr,nbl,nbu,nbd
0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
119995,49,49,43,0.0,0.0,0.0,0.0,0.0,0.0
119996,49,49,44,0.0,0.0,0.0,0.0,0.0,0.0
119997,49,49,45,0.0,0.0,0.0,0.0,0.0,0.0
119998,49,49,46,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
complete_df_with_neighbours_optimized2(species1)

,x,y,hour,building_height,emission_value,nbr,nbl,nbu,nbd
0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
119995,49,49,43,0.0,0.0,0.0,0.0,0.0,0.0
119996,49,49,44,0.0,0.0,0.0,0.0,0.0,0.0
119997,49,49,45,0.0,0.0,0.0,0.0,0.0,0.0
119998,49,49,46,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
%lprun -f complete_df_with_neighbours_optimized2_profiled -f get_neighbours_optimized2_profiled complete_df_with_neighbours_optimized2_profiled(species1,ds)


In [76]:
%lprun -f complete_df_with_neighbours_optimized2 -f get_neighbours_optimized2 complete_df_with_neighbours_optimized2(species1)


### Best implementation with parallel computing

In [10]:
def split_to_chunks(df, num_processes = 4):
    hour_size = len(set(df["hour"]))
    chunk_size = len(set(df["x"]))*len(set(df["y"]))*hour_size//num_processes
    number_of_chunks = df.shape[0]//chunk_size
    print(hour_size,chunk_size,number_of_chunks)
    result = []
    
    for i in range(number_of_chunks):
        result.append(df.iloc[i*chunk_size:(i+1)*chunk_size].copy())
        
    return result

In [25]:
import pandas as pd
import multiprocessing
from concurrent.futures import ThreadPoolExecutor
import joblib
import os

def process_chunk(chunk,df):
    try:
        #print("i am here")
        xs = set(chunk["x"])
        ys = set(chunk["y"])
        result = []
        for x in xs:
            for y in ys:
                df_cell = get_neighbours_optimized2(df, x, y)
                result.append(df_cell)
        return pd.concat(result, ignore_index=True)
    except Exception as e:
        print("error in process",e,multiprocessing.current_process())


def complete_df_with_neighbours_optimized2_parallel(df, num_processes=4): 
    
     
    
    chunks = split_to_chunks(df, num_processes)
    #print(chunks)
    results = []
    result = joblib.Parallel(n_jobs=num_processes)(joblib.delayed(process_chunk)(chunks[i],df) for i in range(len(chunks)))
    #print((result))
    result_df = pd.concat(result)
    return result_df



In [12]:
species1 = pd.read_csv("../data/result_species_500/result1_species_in.csv",sep=";")
species1 = species1[(species1["x"]<200) & (species1["y"]<200)]
species1

,x,y,hour,building_height,emission_value,nbr,nbl,nbu,nbd
0,0,0,0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1,0,0,1,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,0,0,2,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,0,0,3,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,0,0,4,0.000000,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
4785595,199,199,43,21.144001,0.001589,0.0,0.0,0.003678,0.001589
4785596,199,199,44,21.144001,0.000553,0.0,0.0,0.001279,0.000553
4785597,199,199,45,21.144001,0.000286,0.0,0.0,0.000662,0.000286
4785598,199,199,46,21.144001,0.000000,0.0,0.0,0.000000,0.000000


In [59]:
split_to_chunks(species1, 4)

48 76800 4


[         x   y  hour  building_height  emission_value  nbr  nbl  nbu  nbd
 0        0   0     0              0.0             0.0  0.0  0.0  0.0  0.0
 1        0   0     1              0.0             0.0  0.0  0.0  0.0  0.0
 2        0   0     2              0.0             0.0  0.0  0.0  0.0  0.0
 3        0   0     3              0.0             0.0  0.0  0.0  0.0  0.0
 4        0   0     4              0.0             0.0  0.0  0.0  0.0  0.0
 ...     ..  ..   ...              ...             ...  ...  ...  ...  ...
 459835  19  79    43              0.0             0.0  0.0  0.0  0.0  0.0
 459836  19  79    44              0.0             0.0  0.0  0.0  0.0  0.0
 459837  19  79    45              0.0             0.0  0.0  0.0  0.0  0.0
 459838  19  79    46              0.0             0.0  0.0  0.0  0.0  0.0
 459839  19  79    47              0.0             0.0  0.0  0.0  0.0  0.0
 
 [76800 rows x 9 columns],
          x   y  hour  building_height  emission_value  nbr  nbl  nbu  

In [16]:
res = complete_df_with_neighbours_optimized2(species1)
res

,x,y,hour,building_height,emission_value,nbr,nbl,nbu,nbd
0,0,0,0,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,0,0,1,0.000000,0.000000,0.0,0.0,0.000000,0.0
2,0,0,2,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,0,0,3,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,0,0,4,0.000000,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
1919995,199,199,43,21.144001,0.001589,0.0,0.0,0.003678,0.0
1919996,199,199,44,21.144001,0.000553,0.0,0.0,0.001279,0.0
1919997,199,199,45,21.144001,0.000286,0.0,0.0,0.000662,0.0
1919998,199,199,46,21.144001,0.000000,0.0,0.0,0.000000,0.0


In [54]:
joblib.cpu_count()

16

,x,y,hour,building_height,emission_value,nbr,nbl,nbu,nbd
0,0,0,0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0,0,1,0.000000,0.0,0.0,0.0,0.0,0.0
2,0,0,2,0.000000,0.0,0.0,0.0,0.0,0.0
3,0,0,3,0.000000,0.0,0.0,0.0,0.0,0.0
4,0,0,4,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
187,3,3,43,7.131122,0.0,0.0,0.0,0.0,0.0
188,3,3,44,7.131122,0.0,0.0,0.0,0.0,0.0
189,3,3,45,7.131122,0.0,0.0,0.0,0.0,0.0
190,3,3,46,7.131122,0.0,0.0,0.0,0.0,0.0


# Reading .nc files

## Chemistry

In [38]:
ds = nc.Dataset("../data/photoshade_6km10m_lod2_chemistry.nc")

## Static

In [14]:
ds_static = nc.Dataset("../data/photoshade_6km10m_lod2_static.nc")
buildings = pd.DataFrame(ds_static["buildings_2d"][:,:].data)
buildings = buildings.melt(var_name = "x",value_name = "building_height")
buildings["y"] = [i%2440 for i in range(buildings.shape[0])]
new_cols_order = ["x","y","building_height"]
buildings = buildings[new_cols_order]
buildings.replace(-9999,0,inplace=True) # because strange values where there are no pollutants
buildings

,x,y,building_height
0,0,0,0.000000
1,0,1,7.494203
2,0,2,9.762539
3,0,3,7.645858
4,0,4,0.000000
...,...,...,...
451579,671,179,18.275076
451580,671,180,18.620945
451581,671,181,0.000000
451582,671,182,0.000000


# Save information as CSV file

### DF concatenation

In [34]:
result_nspecies = pd.DataFrame()
xrange = [i for i in range(100,120,1)] # Subsetting specific grid part
yrange = [i for i in range(100,120,1)] # Subsetting specific grid part
for i in range(48):
    pd_ihour_nspecies = pd.DataFrame(ds["emission_values"][i,0,yrange,xrange,0].data) #[hour,z,y,x,species]
    pd_ihour_nspecies.index = yrange
    pd_ihour_nspecies.columns = xrange
    pd_ihour_nspecies = pd.melt(pd_ihour_nspecies,var_name = "x",value_name = "emission_value")
    pd_ihour_nspecies["y"] = [i%len(yrange)+yrange[0] for i in range(pd_ihour_nspecies.shape[0])]
    pd_ihour_nspecies["hour"] = i
    #pd_ihour_nspecies = pd_ihour_nspecies.sort_values(["x","y"])
    #buildings = buildings.sort_values(["x","y"])
    #pd_ihour_nspecies["building_height"] = buildings["building_height"]
    #pd_ihour_nspecies=pd.merge(pd_ihour_nspecies,buildings, how= "inner",on = ["x","y"])
    result_nspecies= pd.concat([result_nspecies,pd_ihour_nspecies])

result_nspecies=pd.merge(result_nspecies,buildings, how= "inner",on = ["x","y"])

In [35]:
#result_nspecies["building_height"] = buildings.loc[(buildings["x"]<50) & (buildings["y"]<50)]["building_height"]
new_cols_order = ["x","y","hour","building_height","emission_value"]
result_nspecies= result_nspecies[new_cols_order]
result_nspecies

,x,y,hour,building_height,emission_value
0,101,100,0,0.0,0.0
1,101,100,1,0.0,0.0
2,101,100,2,0.0,0.0
3,101,100,3,0.0,0.0
4,101,100,4,0.0,0.0
...,...,...,...,...,...
5755,119,119,43,0.0,0.0
5756,119,119,44,0.0,0.0
5757,119,119,45,0.0,0.0
5758,119,119,46,0.0,0.0


# PALM Output files

### Download

In [ ]:
output_emissions_no2_0 = nc.Dataset("../data/photoshade_6km10m_lod2_av_masked_M01.000.nc")
output_emissions_no2_1 = nc.Dataset("../data/photoshade_6km10m_lod2_av_masked_M01.001.nc")
output_emissions_no2_2 = nc.Dataset("../data/photoshade_6km10m_lod2_av_masked_M01.002.nc")
output_emissions_no2_3 = nc.Dataset("../data/photoshade_6km10m_lod2_av_masked_M01.003.nc")
output_emissions_no2_4 = nc.Dataset("../data/photoshade_6km10m_lod2_av_masked_M01.004.nc")

output_emissions_no2_list = [output_emissions_no2_0,
                             output_emissions_no2_1,
                             output_emissions_no2_2,
                             output_emissions_no2_3,
                             output_emissions_no2_4,]

In [ ]:
output_emissions_no2_0

### Need to reformat to match input emissions grid dataframe

In [ ]:
result_1species_output = pd.DataFrame()
xrange = [i for i in range(100,180,1)] # Subsetting specific grid part
yrange = [i for i in range(100,180,1)] # Subsetting specific grid part
pollutant = "kc_NO2"
for output_table in output_emissions_no2_list:
    timebins = (output_table["time"][:].data/3600).astype(int) # getting hours array (after conversion from seconds to hours)
    for i in range(len(timebins)):
        pd_ihour_1species = pd.DataFrame(output_table[pollutant][i,0,yrange,xrange].data)#(netcdf format [hour,y,x])
        pd_ihour_1species.index = yrange
        pd_ihour_1species.columns = xrange
        pd_ihour_1species = pd.melt(pd_ihour_1species,var_name = "x",value_name = "emission_value")
        pd_ihour_1species["y"] = [i%len(yrange)+yrange[0] for i in range(pd_ihour_1species.shape[0])]
        pd_ihour_1species["hour"] = timebins[i]-1
        result_1species_output= pd.concat([result_1species_output,pd_ihour_1species])

In [ ]:
result_1species_output.replace(-9999,0,inplace=True) # because strange values where there are no pollutants
result_1species_output = result_1species_output[["x","y","hour","emission_value"]]
result_1species_output

# Plot pollution distribution

In [ ]:
def make_heatmap_from_pd(result_dataframe,hour,target_name = "emission_value"):
    result_1specieshour = result_dataframe.loc[result_dataframe["hour"] == hour]
    result_1specieshour = result_1specieshour[["x","y",target_name]]
    x = result_1specieshour["x"].max() - result_1specieshour["x"].min() + 1
    y = result_1specieshour["y"].max() - result_1specieshour["y"].min() + 1
    v = result_1specieshour[target_name].to_numpy().reshape((x, y))
    indices = result_1specieshour[["x", 'y']].to_numpy().T
    indices[0] = indices[0] - indices[0].min()
    indices[1] = indices[1] - indices[1].min()

    v = np.tile(np.nan, indices.max(axis=1)+1)
    v[tuple(indices)] = result_1specieshour[target_name]
    ax = sns.heatmap(v)
    ax.invert_yaxis()
    plt.show()

In [ ]:
#plotting
hour = 36
plt.subplot(2,1,1)
make_heatmap_from_pd(result_nspecies,hour,target_name="emission_value")
#plt.subplot(2,1,2)
#make_heatmap_from_pd(result_1species_output,hour,target_name="emission_value")

In [ ]:
b_height = ds_static["buildings_2d"][:,:].data
b_height[b_height == -9999] = 0
ax = sns.heatmap(b_height)
plt.show()

In [ ]:
make_heatmap_from_pd(result_nspecies,hour,target_name="building_height")

In [ ]:
#Plot ranges
xrange = [i for i in range(0,670,1)]
yrange = [i for i in range(0,670,1)]
#Data preparing
in_data = ds["emission_values"][0,0,yrange,xrange,1].data          #[hour,z,y,x,nspecies]
out_data = pd.DataFrame(output_emissions_no2_3["kc_NO"][0,0,yrange,xrange].data) #[hour,ku_surf_above,y,x]
out_data[out_data ==-9999] = 0
#plotting
plt.subplot(2,1,1)
ax = sns.heatmap(in_data)
plt.show()
plt.subplot(2,1,2)
ax = sns.heatmap(out_data)
plt.show()

# Outputing relevant information in csvs

In [ ]:
completed_1species = complete_df_with_neighbours(result_nspecies)
completed_1species

In [ ]:
completed_1species_opt = complete_df_with_neighbours_optimized2(result_nspecies)
completed_1species_opt

In [ ]:
completed_1species_out = complete_df_with_neighbours(result_1species_output)
completed_1species_out

In [ ]:
completed_1species.to_csv("../data/result600-800in.csv",index=False,sep =";",encoding='utf-8', float_format='%.15f')
#completed_1species_out.to_csv("../data/result1_species_out.csv",index=False,sep =";",encoding='utf-8', float_format='%.15f')


In [23]:
import time
source_array = [i for i in range(int(20e6))]
target_array = []
start = time.time_ns()

for i in range(int(20e6)):
    target_array.append(source_array[-i])
    target_array.append(source_array[-i])
    target_array.append(source_array[-i])
    target_array.append(source_array[-i])
end = time.time_ns()

print((end-start)/1e9)

6.5523349
